In [1]:
import IPython
import pandas as pd
import numpy as np


In [2]:
ORIGINAL_DF = pd.read_csv("diabetic_data.csv",index_col=0, na_values='?', low_memory=False)
df = ORIGINAL_DF.copy(deep=True)


In [3]:
df.head()

,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
encounter_id,,,,,,,,,,,,,,,,,,,,,
2278392,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,NaN,...,No,No,No,No,No,No,No,No,No,NO
149190,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,NaN,...,No,Up,No,No,No,No,No,Ch,Yes,>30
64410,86047875,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,NaN,...,No,No,No,No,No,No,No,No,Yes,NO
500364,82442376,Caucasian,Male,[30-40),NaN,1,1,7,2,NaN,...,No,Up,No,No,No,No,No,Ch,Yes,NO
16680,42519267,Caucasian,Male,[40-50),NaN,1,1,7,1,NaN,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [4]:
df.groupby('readmitted').count()["patient_nbr"]# quick to see distro of output
# <30 most interesting, will need to upsample, probs

readmitted
<30    11357
>30    35545
NO     54864
Name: patient_nbr, dtype: int64

In [5]:
missingvalues = pd.DataFrame(df.isnull().sum(axis=0), columns = ['number_of_missing_values'])
missingvalues['feature'] = missingvalues.index
missingvalues = missingvalues[['feature','number_of_missing_values']].reset_index (drop = True)
missingvalues[missingvalues.number_of_missing_values > 0]

,feature,number_of_missing_values
1,race,2273
4,weight,98569
9,payer_code,40256
10,medical_specialty,49949
17,diag_1,21
18,diag_2,358
19,diag_3,1423


In [6]:
#Dropping massive NA vars
df = df.drop(['weight', 'payer_code', 'medical_specialty'], axis = 1)

In [7]:
categ = []
numer = []
for col in df.columns:
    if df[col].dtype == object and col != "patient_nbr":
        categ.append(col)
    else:
        numer.append(col)
print(categ)
print(numer)

['race', 'gender', 'age', 'diag_1', 'diag_2', 'diag_3', 'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'insulin', 'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted']
['patient_nbr', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id', 'time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']


In [8]:
#Counting categorical modalities:
categNoDiag = list(filter(lambda x: x != 'diag_1' and x != 'diag_2' and x != 'diag_3', categ))
for name in categNoDiag:
    print(df.groupby(name).count()["patient_nbr"])

race
AfricanAmerican    19210
Asian                641
Caucasian          76099
Hispanic            2037
Other               1506
Name: patient_nbr, dtype: int64
gender
Female             54708
Male               47055
Unknown/Invalid        3
Name: patient_nbr, dtype: int64
age
[0-10)        161
[10-20)       691
[20-30)      1657
[30-40)      3775
[40-50)      9685
[50-60)     17256
[60-70)     22483
[70-80)     26068
[80-90)     17197
[90-100)     2793
Name: patient_nbr, dtype: int64
max_glu_serum
>200     1485
>300     1264
None    96420
Norm     2597
Name: patient_nbr, dtype: int64
A1Cresult
>7       3812
>8       8216
None    84748
Norm     4990
Name: patient_nbr, dtype: int64
metformin
Down        575
No        81778
Steady    18346
Up         1067
Name: patient_nbr, dtype: int64
repaglinide
Down          45
No        100227
Steady      1384
Up           110
Name: patient_nbr, dtype: int64
nateglinide
Down          11
No        101063
Steady       668
Up            24
Name: pati

In [9]:
df["gender"]=df["gender"].replace("Unknown/Invalid", np.NaN)

In [10]:
#Re-encoding variables based on expert's study online:
df['admission_type_id'] = df['admission_type_id'].replace(2,1)
df['admission_type_id'] = df['admission_type_id'].replace(7,1)
df['admission_type_id'] = df['admission_type_id'].replace(6,5)
df['admission_type_id'] = df['admission_type_id'].replace(8,5)

df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(6,1)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(8,1)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(9,1)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(13,1)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(3,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(4,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(5,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(14,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(22,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(23,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(24,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(12,10)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(15,10)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(16,10)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(17,10)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(25,18)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(26,18)

df['admission_source_id'] = df['admission_source_id'].replace(2,1)
df['admission_source_id'] = df['admission_source_id'].replace(3,1)
df['admission_source_id'] = df['admission_source_id'].replace(5,4)
df['admission_source_id'] = df['admission_source_id'].replace(6,4)
df['admission_source_id'] = df['admission_source_id'].replace(10,4)
df['admission_source_id'] = df['admission_source_id'].replace(22,4)
df['admission_source_id'] = df['admission_source_id'].replace(25,4)
df['admission_source_id'] = df['admission_source_id'].replace(15,9)
df['admission_source_id'] = df['admission_source_id'].replace(17,9)
df['admission_source_id'] = df['admission_source_id'].replace(20,9)
df['admission_source_id'] = df['admission_source_id'].replace(21,9)
df['admission_source_id'] = df['admission_source_id'].replace(13,11)
df['admission_source_id'] = df['admission_source_id'].replace(14,11)

#To avoid dropping rows which have NA in diag2, or diag3 (not truly NA, just not as many diags), we'll replace NAs with -1 for later

df['diag_2'] = df['diag_2'].replace(np.NaN, -1)
df['diag_3'] = df['diag_3'].replace(np.NaN, -1)


In [11]:
#Drop dead people (obviously not interesting for the study:
df["discharge_disposition_id"]=df["discharge_disposition_id"].replace(11, np.NaN)

In [12]:
print(ORIGINAL_DF.shape)
df.dropna(inplace = True)
print(df.shape) #not so many, :D

(101766, 49)
(97865, 46)


In [13]:
#Categorical variable encoding:
def encode(colname, zeroes):
    df[colname] = df[colname].apply(lambda x: 0 if x in zeroes else 1)
    
meds = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'glipizide', 'glyburide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'insulin', 'glyburide-metformin', 'tolazamide', 'metformin-pioglitazone','metformin-rosiglitazone', 'glimepiride-pioglitazone', 'glipizide-metformin', 'troglitazone', 'tolbutamide', 'acetohexamide',"examide","citoglipton"]
for med in meds:
    encode(med, ["No"])

#For these 2, >x is a 'positive', Norm is normal and None means no test was performed (NA)
df['A1Cresult'] = df['A1Cresult'].replace('>7', 1)
df['A1Cresult'] = df['A1Cresult'].replace('>8', 1)
df['A1Cresult'] = df['A1Cresult'].replace('Norm', 0)
df['A1Cresult'] = df['A1Cresult'].replace('None', -99)

df['max_glu_serum'] = df['max_glu_serum'].replace('>200', 1)
df['max_glu_serum'] = df['max_glu_serum'].replace('>300', 1)
df['max_glu_serum'] = df['max_glu_serum'].replace('Norm', 0)
df['max_glu_serum'] = df['max_glu_serum'].replace('None', -99)



In [14]:
#Using online info, we know how to semantically group diags
df['temp_diag1'] = df['diag_1']
df['temp_diag2'] = df['diag_2']
df['temp_diag3'] = df['diag_3']


df.loc[df['diag_1'].str.contains('V', na=True), ['temp_diag1']] = 0 # there are no NAs, just a quick hack
df.loc[df['diag_1'].str.contains('E', na=True), ['temp_diag1']] = 0
df.loc[df['diag_2'].str.contains('V', na=True), ['temp_diag2']] = 0
df.loc[df['diag_2'].str.contains('E', na=True), ['temp_diag2']] = 0
df.loc[df['diag_3'].str.contains('V', na=True), ['temp_diag3']] = 0
df.loc[df['diag_3'].str.contains('E', na=True), ['temp_diag3']] = 0

df['temp_diag1'] = df['temp_diag1'].astype(float)
df['temp_diag2'] = df['temp_diag2'].astype(float)
df['temp_diag3'] = df['temp_diag3'].astype(float)



for index, row in df.iterrows():
    if (row['temp_diag1'] >= 390 and row['temp_diag1'] < 460) or (np.floor(row['temp_diag1']) == 785):
        df.loc[index, 'temp_diag1'] = 1
    elif (row['temp_diag1'] >= 460 and row['temp_diag1'] < 520) or (np.floor(row['temp_diag1']) == 786):
        df.loc[index, 'temp_diag1'] = 2
    elif (row['temp_diag1'] >= 520 and row['temp_diag1'] < 580) or (np.floor(row['temp_diag1']) == 787):
        df.loc[index, 'temp_diag1'] = 3
    elif (np.floor(row['temp_diag1']) == 250):
        df.loc[index, 'temp_diag1'] = 4
    elif (row['temp_diag1'] >= 800 and row['temp_diag1'] < 1000):
        df.loc[index, 'temp_diag1'] = 5
    elif (row['temp_diag1'] >= 710 and row['temp_diag1'] < 740):
        df.loc[index, 'temp_diag1'] = 6
    elif (row['temp_diag1'] >= 580 and row['temp_diag1'] < 630) or (np.floor(row['temp_diag1']) == 788):
        df.loc[index, 'temp_diag1'] = 7
    elif (row['temp_diag1'] >= 140 and row['temp_diag1'] < 240):
        df.loc[index, 'temp_diag1'] = 8
    else:
        df.loc[index, 'temp_diag1'] = 0
        
    if (row['temp_diag2'] >= 390 and row['temp_diag2'] < 460) or (np.floor(row['temp_diag2']) == 785):
        df.loc[index, 'temp_diag2'] = 1
    elif (row['temp_diag2'] >= 460 and row['temp_diag2'] < 520) or (np.floor(row['temp_diag2']) == 786):
        df.loc[index, 'temp_diag2'] = 2
    elif (row['temp_diag2'] >= 520 and row['temp_diag2'] < 580) or (np.floor(row['temp_diag2']) == 787):
        df.loc[index, 'temp_diag2'] = 3
    elif (np.floor(row['temp_diag2']) == 250):
        df.loc[index, 'temp_diag2'] = 4
    elif (row['temp_diag2'] >= 800 and row['temp_diag2'] < 1000):
        df.loc[index, 'temp_diag2'] = 5
    elif (row['temp_diag2'] >= 710 and row['temp_diag2'] < 740):
        df.loc[index, 'temp_diag2'] = 6
    elif (row['temp_diag2'] >= 580 and row['temp_diag2'] < 630) or (np.floor(row['temp_diag2']) == 788):
        df.loc[index, 'temp_diag2'] = 7
    elif (row['temp_diag2'] >= 140 and row['temp_diag2'] < 240):
        df.loc[index, 'temp_diag2'] = 8
    else:
        df.loc[index, 'temp_diag2'] = 0
    
    if (row['temp_diag3'] >= 390 and row['temp_diag3'] < 460) or (np.floor(row['temp_diag3']) == 785):
        df.loc[index, 'temp_diag3'] = 1
    elif (row['temp_diag3'] >= 460 and row['temp_diag3'] < 520) or (np.floor(row['temp_diag3']) == 786):
        df.loc[index, 'temp_diag3'] = 2
    elif (row['temp_diag3'] >= 520 and row['temp_diag3'] < 580) or (np.floor(row['temp_diag3']) == 787):
        df.loc[index, 'temp_diag3'] = 3
    elif (np.floor(row['temp_diag3']) == 250):
        df.loc[index, 'temp_diag3'] = 4
    elif (row['temp_diag3'] >= 800 and row['temp_diag3'] < 1000):
        df.loc[index, 'temp_diag3'] = 5
    elif (row['temp_diag3'] >= 710 and row['temp_diag3'] < 740):
        df.loc[index, 'temp_diag3'] = 6
    elif (row['temp_diag3'] >= 580 and row['temp_diag3'] < 630) or (np.floor(row['temp_diag3']) == 788):
        df.loc[index, 'temp_diag3'] = 7
    elif (row['temp_diag3'] >= 140 and row['temp_diag3'] < 240):
        df.loc[index, 'temp_diag3'] = 8
    else:
        df.loc[index, 'temp_diag3'] = 0

In [15]:
df.head().T

encounter_id,2278392,149190,64410,500364,16680
patient_nbr,8222157,55629189,86047875,82442376,42519267
race,Caucasian,Caucasian,AfricanAmerican,Caucasian,Caucasian
gender,Female,Female,Female,Male,Male
age,[0-10),[10-20),[20-30),[30-40),[40-50)
admission_type_id,5,1,1,1,1
discharge_disposition_id,18,1,1,1,1
admission_source_id,1,7,7,7,7
time_in_hospital,1,3,2,2,1
num_lab_procedures,41,59,11,44,51
num_procedures,0,0,5,1,0


In [16]:
#Dropping old vars
df = df.drop(["patient_nbr"], axis=1)
df = df.drop(["diag_1","diag_2","diag_3"], axis=1)

In [17]:
#Dealing with rest of categoricals
df['age'] = df['age'].replace('[0-10)', 0)
df['age'] = df['age'].replace('[10-20)', 1)
df['age'] = df['age'].replace('[20-30)', 2)
df['age'] = df['age'].replace('[30-40)', 3)
df['age'] = df['age'].replace('[40-50)', 4)
df['age'] = df['age'].replace('[50-60)', 5)
df['age'] = df['age'].replace('[60-70)', 6)
df['age'] = df['age'].replace('[70-80)', 7)
df['age'] = df['age'].replace('[80-90)', 8)
df['age'] = df['age'].replace('[90-100)', 9)

from sklearn.preprocessing import LabelEncoder
categoricals = ["race","gender","change", "diabetesMed"]

le = LabelEncoder()
for vname in categoricals:
    df[vname] = le.fit_transform(df[vname])
    
df['readmitted'] = df['readmitted'].apply(lambda x: 0 if x == "<30" else 1)

In [18]:
#df['readmitted_test'] = df['readmitted']
df.groupby('readmitted').count()["race"]# quick to see distro of output

#df['readmitted_test'] = df['readmitted_test'].apply(lambda x: 0 if x == "<30" else 1)
#df.groupby('readmitted_test').count()["race"]

readmitted
0    11164
1    86701
Name: race, dtype: int64

In [19]:
df.head().T #All numerical Data

encounter_id,2278392,149190,64410,500364,16680
race,2.0,2.0,0.0,2.0,2.0
gender,0.0,0.0,0.0,1.0,1.0
age,0.0,1.0,2.0,3.0,4.0
admission_type_id,5.0,1.0,1.0,1.0,1.0
discharge_disposition_id,18.0,1.0,1.0,1.0,1.0
admission_source_id,1.0,7.0,7.0,7.0,7.0
time_in_hospital,1.0,3.0,2.0,2.0,1.0
num_lab_procedures,41.0,59.0,11.0,44.0,51.0
num_procedures,0.0,0.0,5.0,1.0,0.0
num_medications,1.0,18.0,13.0,16.0,8.0


In [20]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
race,97865.0,1.658734,0.861971,0.0,2.0,2.0,2.0,4.0
gender,97865.0,0.461074,0.498485,0.0,0.0,0.0,1.0,1.0
age,97865.0,6.081868,1.595037,0.0,5.0,6.0,7.0,9.0
admission_type_id,97865.0,1.789189,1.335100,1.0,1.0,1.0,3.0,5.0
discharge_disposition_id,97865.0,2.102549,3.756034,1.0,1.0,1.0,2.0,28.0
admission_source_id,97865.0,5.135074,2.880157,1.0,1.0,7.0,7.0,11.0
time_in_hospital,97865.0,4.391713,2.976420,1.0,2.0,4.0,6.0,14.0
num_lab_procedures,97865.0,42.919726,19.640470,1.0,31.0,44.0,57.0,132.0
num_procedures,97865.0,1.331978,1.698432,0.0,0.0,1.0,2.0,6.0
num_medications,97865.0,15.986624,8.084550,1.0,10.0,15.0,20.0,81.0


In [21]:
#Examide and citogliptol are all 0, so drop. Same for metformin-rosiglitazone
df = df.drop(["examide","citoglipton","metformin-rosiglitazone"], axis=1)

In [22]:
#TODO: kurtosis, skew:
numericInterestingVariables =['age', 'time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']
for var in numericInterestingVariables:
    if (abs(df[var].skew()) >2) & (abs(df[var].kurtosis()) >2):
        print(var, " needs log")
        df[var] =  np.log1p(df[var])


number_outpatient  needs log
number_emergency  needs log
number_inpatient  needs log


In [23]:
print(df.shape)
print(df.info())
print(df.describe().T)

(97865, 42)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 97865 entries, 2278392 to 443867222
Data columns (total 42 columns):
race                        97865 non-null int64
gender                      97865 non-null int64
age                         97865 non-null int64
admission_type_id           97865 non-null int64
discharge_disposition_id    97865 non-null float64
admission_source_id         97865 non-null int64
time_in_hospital            97865 non-null int64
num_lab_procedures          97865 non-null int64
num_procedures              97865 non-null int64
num_medications             97865 non-null int64
number_outpatient           97865 non-null float64
number_emergency            97865 non-null float64
number_inpatient            97865 non-null float64
number_diagnoses            97865 non-null int64
max_glu_serum               97865 non-null int64
A1Cresult                   97865 non-null int64
metformin                   97865 non-null int64
repaglinide                 

In [24]:
from sklearn.model_selection import train_test_split
inputSet = df.drop("readmitted", axis=1)
outputSet = df["readmitted"]

X_train, X_test, y_train, y_test = train_test_split(inputSet, outputSet, random_state=42,test_size=0.3)


In [25]:
print(y_train.describe())

count    68505.000000
mean         0.886797
std          0.316844
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: readmitted, dtype: float64


In [26]:
# Move Smote here?
from imblearn import over_sampling as os

smoter = os.SMOTE(random_state = 42)
X_train, y_train = smoter.fit_sample(X_train, y_train)

ModuleNotFoundError: No module named 'imblearn'

In [28]:
from collections import Counter
print('New dataset shape {}'.format(Counter(y_train)))

New dataset shape Counter({1: 60750, 0: 7755})


In [29]:
TRAIN = pd.DataFrame(X_train, columns = list(inputSet))
TRAIN["TARGET"] = y_train
TRAIN.to_csv('./TRAIN.csv')

In [30]:
TEST = X_test
TEST["TARGET"] = y_test
TEST.to_csv('./TEST.csv')

/opt/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
